# Trabalho 2

### Importando as bibliotecas que serão usadas no script inteiro

In [32]:
import math
import pandas  as pd
import numpy   as np

import matplotlib.pyplot as plt

from sklearn.model_selection  import train_test_split
from sklearn.metrics          import mean_squared_error, r2_score
from sklearn.preprocessing    import StandardScaler, MinMaxScaler, PolynomialFeatures, LabelBinarizer

from sklearn.linear_model     import LinearRegression, Ridge, Lasso, SGDRegressor
from sklearn.neighbors        import KNeighborsRegressor

from scipy.stats              import pearsonr

### Inserindo os dados do conjunto de treinamento e teste em variáveis

In [33]:
dados = pd.read_csv("conjunto_de_treinamento.csv")

### Separando os dados do conjunto de treinamento em treinamento e teste

In [34]:
x = dados.iloc[:,1:-1].to_numpy()
y = dados.iloc[:,-1].to_numpy()

print(x)
print("---------------------------")
print(y)

# 4683 total de amostras

x_treino, y_treino, x_teste, y_teste = train_test_split(
    x,
    y,
    train_size = 3500,
    random_state = 0
)

# testar com e sem escala

# escala = StandardScaler()
# escala.fit(x_treino)

# x_treino = escala.transform(x_treino)
# x_teste  = escala.transform(x_teste)

[['Casa' 'Imbiribeira' 'Imobiliaria' ... 0 0 0]
 ['Apartamento' 'Casa Amarela' 'Imobiliaria' ... 0 0 0]
 ['Apartamento' 'Encruzilhada' 'Imobiliaria' ... 0 0 0]
 ...
 ['Apartamento' 'Torre' 'Imobiliaria' ... 0 0 0]
 ['Apartamento' 'Casa Amarela' 'Imobiliaria' ... 0 0 0]
 ['Apartamento' 'Madalena' 'Imobiliaria' ... 0 0 0]]
---------------------------
[1000000.  680000.  450000. ...  252000.  780000.  577035.]


Verificando as variáveis e a quantidade de cada uma para fazer o pré-processamento dos dados

In [35]:
for v in dados:
    print("\n%15s: " %v, "%4d categorias" % len(dados[v].unique() ) )
    print(dados[v].unique(), "\n")

# variaveis string: tipo (4), bairro (66), tipo_vendedor (2), diferenciais (83)


             Id:  4683 categorias
[2000 2001 2002 ... 6680 6681 6682] 


           tipo:     4 categorias
['Casa' 'Apartamento' 'Quitinete' 'Loft'] 


         bairro:    66 categorias
['Imbiribeira' 'Casa Amarela' 'Encruzilhada' 'Boa Viagem' 'Rosarinho'
 'Boa Vista' 'Espinheiro' 'Tamarineira' 'Gracas' 'Madalena' 'Parnamirim'
 'S Jose' 'Setubal' 'Arruda' 'Pina' 'Beira Rio' 'Caxanga' 'Casa Forte'
 'Prado' 'Iputinga' 'Campo Grande' 'Dois Irmaos' 'Torreao'
 'Ilha do Retiro' 'Areias' 'Varzea' 'Cordeiro' 'Torre' 'Santana' 'Barro'
 'Poco da Panela' 'Ipsep' 'Aflitos' 'Poco' 'Apipucos' 'Derby'
 'Cid Universitaria' 'Bongi' 'Jaqueira' 'Sto Amaro' 'Tejipio' 'Recife'
 'Monteiro' 'Macaxeira' 'Sancho' 'Afogados' 'San Martin' 'Cajueiro'
 'Hipodromo' 'Guabiraba' 'Engenho do Meio' 'Piedade' 'Jd S Paulo'
 'Lagoa do Araca' 'Ponto de Parada' 'Ilha do Leite' 'Estancia' 'Paissandu'
 'Zumbi' 'Agua Fria' 'Benfica' 'Soledade' 'Centro' 'Sto Antonio' 'Coelhos'
 'Cohab'] 


  tipo_vendedor:     2 categorias
['I

Verificar a quantidade de amostras de cada classe

In [36]:
print(dados["tipo_vendedor"].value_counts())
print(dados["tipo"].value_counts())
print(dados["bairro"].value_counts())
print(dados["diferenciais"].value_counts())

tipo_vendedor
Imobiliaria      4556
Pessoa Fisica     127
Name: count, dtype: int64
tipo
Apartamento    4501
Casa            177
Loft              3
Quitinete         2
Name: count, dtype: int64
bairro
Boa Viagem         1599
Madalena            327
Torre               240
Casa Amarela        225
Espinheiro          208
                   ... 
Ponto de Parada       1
Centro                1
Sto Antonio           1
Coelhos               1
Cohab                 1
Name: count, Length: 66, dtype: int64
diferenciais
piscina e churrasqueira                    1340
nenhum                                      986
piscina e sauna                             350
piscina e playground                        338
salao de festas                             187
                                           ... 
children care                                 1
sauna e quadra poliesportiva                  1
piscina e quadra de squash                    1
esquina e quadra poliesportiva                1
cam

Verificar a correlação de pearson entre os parâmetros e o alvo (obs: os parâmetros não parecem muito promissores, mas veremos mais na hora de treinar o modelo):

In [ ]:
colunas = dados.columns
for col in colunas:
    if type(dados[col][0]) != str:
        corr = pearsonr(dados[col], dados['preco'])
        print(col, " = %.4f" % corr[0], " , %.10f" % corr[1])

Fazer o one hot encoding e a binarização das variáveis 

In [28]:
dados = pd.get_dummies(dados, columns = ["tipo"])

print(dados.head(2).T)

print("-"*100)

binarizador = LabelBinarizer()

for v in ["tipo_vendedor"]:
    dados[v] = binarizador.fit_transform(dados[v])

print(dados.head(2).T)

                               0                        1
Id                          2000                     2001
bairro               Imbiribeira             Casa Amarela
tipo_vendedor        Imobiliaria              Imobiliaria
quartos                        3                        4
suites                         3                        4
vagas                          5                        2
area_util                    223                      157
area_extra                   167                        0
diferenciais      piscina e copa  piscina e churrasqueira
churrasqueira                  0                        1
estacionamento                 0                        0
piscina                        1                        1
playground                     0                        0
quadra                         0                        0
s_festas                       0                        0
s_jogos                        0                        0
s_ginastica   

### Regressor Linear

In [ ]:
print('')
print("REGRESSOR LINEAR:")
print('')

print("Métrica   DENTRO da amostra   FORA da amostra")
print("-------   -----------------   ---------------")

regressor_linear = LinearRegression()

regressor_linear = regressor_linear.fit(x_treino, y_treino)

y_resposta_treino = regressor_linear.predict(x_treino)
y_resposta_teste  = regressor_linear.predict(x_teste)

mse_in  = mean_squared_error(y_treino, y_resposta_treino)
rmse_in = math.sqrt(mse_in)
r2_in   = r2_score(y_treino, y_resposta_treino)
#rmspe_in = (np.sqrt(np.mean(np.square((y_true - y_pred) / y_true)))) * 100

mse_out  = mean_squared_error(y_teste, y_resposta_teste)
rmse_out = math.sqrt(mse_out)
r2_out   = r2_score(y_teste, y_resposta_teste)
#rmspe_out = (np.sqrt(np.mean(np.square((y_true - y_pred) / y_true)))) * 100

print("%7s   %17.4f   %15.4f" % ("mse"  , mse_in  , mse_out))
print("%7s   %17.4f   %15.4f" % ("rmse" , rmse_in , rmse_out))
print("%7s   %17.4f   %15.4f" % ("r2"   , r2_in   , r2_out))

plt.scatter(x = y_teste, y = y_resposta_teste)

### Regressor KNN

In [ ]:
print('')
print(" REGRESSOR KNN:")
print('')

print(" K   DENTRO da amostra   FORA da amostra")
print("---   ----------------   ---------------")

for k in range (1,21):
    
    regressor_knn   = KNeighborsRegressor(
        n_neighbors = k,
        weights     = "uniform"      # uniform ou distance
    )
    
    regressor_knn = regressor_knn.fit(x_treino, y_treino)

    y_resposta_treino = regressor_knn.predict(x_treino)
    y_resposta_teste  = regressor_knn.predict(x_teste)

    print('')
    print(" REGRESSOR KNN:")
    print('')

    print(" K   DENTRO da amostra   FORA da amostra")
    print("---   ----------------   ---------------")

    mse_in  = mean_squared_error(y_treino, y_resposta_treino)
    rmse_in = math.sqrt(mse_in)
    r2_in   = r2_score(y_treino, y_resposta_treino)
    #rmspe_in = (np.sqrt(np.mean(np.square((y_true - y_pred) / y_true)))) * 100
    
    mse_out  = mean_squared_error(y_teste, y_resposta_teste)
    rmse_out = math.sqrt(mse_out)
    r2_out   = r2_score(y_teste, y_resposta_teste)
    #rmspe_out = (np.sqrt(np.mean(np.square((y_true - y_pred) / y_true)))) * 100
    
    print(" %3d   %17.4f   %15.4f" % (k , rmse_in , rmse_out))

### Regressor Polinomial

In [ ]:
print('')
print(" REGRESSOR POLINOMIAL:")
print('')

print(" K   DENTRO da amostra   FORA da amostra")
print("---   ----------------   ---------------")

for k in range (1,6):

    pf = PolynomialFeatures(degrees = k)

    pf = pf.fit(x_treino)

    x_treino_poly = pf.transform(x_treino)
    x_teste_poly  = pf.transform(x_teste)
    
    regressor_linear = LinearRegression()

    regressor_linear = regressor_linear.fit(x_treino_poly, y_treino)

    y_resposta_treino = regressor_linear.predict(x_treino_poly)
    y_resposta_teste  = regressor_linear.predict(x_teste_poly)

    mse_in  = mean_squared_error(y_treino, y_resposta_treino)
    rmse_in = math.sqrt(mse_in)
    r2_in   = r2_score(y_treino, y_resposta_treino)
    #rmspe_in = (np.sqrt(np.mean(np.square((y_true - y_pred) / y_true)))) * 100
    
    mse_out  = mean_squared_error(y_teste, y_resposta_teste)
    rmse_out = math.sqrt(mse_out)
    r2_out   = r2_score(y_teste, y_resposta_teste)
    #rmspe_out = (np.sqrt(np.mean(np.square((y_true - y_pred) / y_true)))) * 100
    
    print(" %3d   %17.4f   %15.4f" % (k , rmse_in , rmse_out))

### Regressão Polinomial com Regularização Ridge

In [ ]:
print('')
print(" REGRESSOR POLINOMIAL COM REGULARIZAÇÃO RIDGE (L2):")
print('')

print(" K   DENTRO da amostra   FORA da amostra")
print("---   ----------------   ---------------")

for k in range (1,6):

    pf = PolynomialFeatures(degrees = k)

    pf = pf.fit(x_treino)

    x_treino_poly = pf.transform(x_treino)
    x_teste_poly  = pf.transform(x_teste)
    
    regressor_ridge = Ridge(alpha = 1.0)   # mudar o alpha para melhorar o modelo

    regressor_ridge = regressor_ridge.fit(x_treino_poly, y_treino)

    y_resposta_treino = regressor_ridge.predict(x_treino_poly)
    y_resposta_teste  = regressor_ridge.predict(x_teste_poly)

    mse_in  = mean_squared_error(y_treino, y_resposta_treino)
    rmse_in = math.sqrt(mse_in)
    r2_in   = r2_score(y_treino, y_resposta_treino)
    #rmspe_in = (np.sqrt(np.mean(np.square((y_true - y_pred) / y_true)))) * 100
    
    mse_out  = mean_squared_error(y_teste, y_resposta_teste)
    rmse_out = math.sqrt(mse_out)
    r2_out   = r2_score(y_teste, y_resposta_teste)
    #rmspe_out = (np.sqrt(np.mean(np.square((y_true - y_pred) / y_true)))) * 100
    
    print(" %3d   %17.4f   %15.4f" % (k , rmse_in , rmse_out))

### Regressão Polinomial com Regularização Lasso

In [ ]:
print('')
print(" REGRESSOR POLINOMIAL COM REGULARIZAÇÃO LASSO (L1):")
print('')

print(" K   DENTRO da amostra   FORA da amostra")
print("---   ----------------   ---------------")

for k in range (1,6):

    pf = PolynomialFeatures(degrees = k)

    pf = pf.fit(x_treino)

    x_treino_poly = pf.transform(x_treino)
    x_teste_poly  = pf.transform(x_teste)
    
    regressor_lasso = Lasso(alpha = 1.0, max_iter = 10000)   # mudar o alpha para melhorar o modelo

    regressor_lasso = regressor_lasso.fit(x_treino_poly, y_treino)

    y_resposta_treino = regressor_lasso.predict(x_treino_poly)
    y_resposta_teste  = regressor_lasso.predict(x_teste_poly)

    mse_in  = mean_squared_error(y_treino, y_resposta_treino)
    rmse_in = math.sqrt(mse_in)
    r2_in   = r2_score(y_treino, y_resposta_treino)
    #rmspe_in = (np.sqrt(np.mean(np.square((y_true - y_pred) / y_true)))) * 100
    
    mse_out  = mean_squared_error(y_teste, y_resposta_teste)
    rmse_out = math.sqrt(mse_out)
    r2_out   = r2_score(y_teste, y_resposta_teste)
    #rmspe_out = (np.sqrt(np.mean(np.square((y_true - y_pred) / y_true)))) * 100
    
    print(" %3d   %17.4f   %15.4f" % (k , rmse_in , rmse_out))

### Regressor SGD

In [ ]:
print('')
print(" REGRESSOR SGD:")
print('')

print("Métrica   DENTRO da amostra   FORA da amostra")
print("-------   -----------------   ---------------")

regressor_sgd = SGDRegressor(loss = "squared_loss",
                             alpha = 1.0,
                             penalty = "l2",
                             tol=1e-5,
                             max_iter = 100000
                            )   # mudar hiperparâmetros para melhorar o modelo

regressor_sgd = regressor_sgd.fit(x_treino, y_treino)

y_resposta_treino = regressor_sgd.predict(x_treino)
y_resposta_teste  = regressor_sgd.predict(x_teste)

mse_in  = mean_squared_error(y_treino, y_resposta_treino)
rmse_in = math.sqrt(mse_in)
r2_in   = r2_score(y_treino, y_resposta_treino)
#rmspe_in = (np.sqrt(np.mean(np.square((y_true - y_pred) / y_true)))) * 100

mse_out  = mean_squared_error(y_teste, y_resposta_teste)
rmse_out = math.sqrt(mse_out)
r2_out   = r2_score(y_teste, y_resposta_teste)
#rmspe_out = (np.sqrt(np.mean(np.square((y_true - y_pred) / y_true)))) * 100

print("%7s   %17.4f   %15.4f" % ("mse"  , mse_in  , mse_out))
print("%7s   %17.4f   %15.4f" % ("rmse" , rmse_in , rmse_out))
print("%7s   %17.4f   %15.4f" % ("r2"   , r2_in   , r2_out))